In [1]:
import pandas as pd

In [2]:
r2 = pd.read_csv('input/rovat_2.csv', na_filter=False, dtype=object, usecols=['ceg_id', 'nev'])

In [13]:
r2[r2.nev.str.lower().str.contains('alatt')].nev.str.slice(-20).head()

12     végelszámolás alatt
17     végelszámolás alatt
22     "felszámolás alatt"
30     "felszámolás alatt"
36    ég felszámolás alatt
Name: nev, dtype: object

In [4]:
r2['doglott'] = r2.nev.str.lower().str.contains('alatt')

In [5]:
g_cegek = r2.groupby(by='ceg_id')

In [6]:
doglott_cegek_mset = g_cegek.filter(lambda x: x.doglott.any())

In [7]:
doglott_cegek = doglott_cegek_mset.ceg_id.drop_duplicates()
cegek = r2.ceg_id.drop_duplicates()

In [8]:
def count_by_mc(ceg_ids):
    mc = ceg_ids.str.slice(0, 2)
    return mc.value_counts().sort_index()

In [9]:
count_by_mc(doglott_cegek)

01     69
03    101
13    125
18     30
Name: ceg_id, dtype: int64

In [10]:
count_by_mc(cegek)

01    548
03    476
13    677
18    199
Name: ceg_id, dtype: int64

In [11]:
c, d = count_by_mc(cegek).align(count_by_mc(doglott_cegek), fill_value=0)
raw_result = pd.DataFrame(dict(cegek=c, doglott=d, arany=d/c)).reset_index().rename(columns={'index': 'bs'})[['bs', 'cegek', 'doglott', 'arany']]
raw_result

,bs,cegek,doglott,arany
0,01,548,69,0.125912
1,03,476,101,0.212185
2,13,677,125,0.184638
3,18,199,30,0.150754


In [12]:
pd.merge(raw_result, pd.read_csv('megye.csv', dtype=object))[['megye', 'cegek', 'doglott', 'arany']]

,megye,cegek,doglott,arany
0,Főváros,548,69,0.125912
1,Bács-Kiskun,476,101,0.212185
2,Pest,677,125,0.184638
3,Vas,199,30,0.150754
